<a href="https://colab.research.google.com/github/rayeeed/UFP_Project/blob/main/PM2_5_Prediction_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
#from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score,cross_val_predict
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeCV
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectFromModel
from time import time
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import seaborn as sns
from sklearn.preprocessing import Normalizer,MinMaxScaler,StandardScaler,RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso, BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import RFE,RFECV
from sklearn.linear_model import (HuberRegressor,
                              	RANSACRegressor, TheilSenRegressor)

In [ ]:
!pip install optuna
import optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.3 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import KBinsDiscretizer

# Load the dataset
df = pd.read_excel('/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/model_35.xlsx', index_col='site_name')

# Drop unnecessary columns
df.drop(['Site_type', 'OID'], inplace=True, axis=1)

# Separate the target variable and features
y = df['Mean PM2.5 (µg/m3)']
X = df.drop(['Mean PNC (# / cm3)', 'Mean PM2.5 (µg/m3)'], axis=1)

In [ ]:
# Discretize the target variable into bins using 'quantile' strategy
binner = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
y_binned = binner.fit_transform(y.values.reshape(-1, 1)).flatten()

# Perform stratified train-test split
split = StratifiedShuffleSplit(n_splits=1, test_size=.2, random_state=42)
for train_index, test_index in split.split(X, y_binned):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Check the distribution of bins in the train and test sets
print("Train bin distribution:\n", pd.value_counts(y_binned[train_index], sort=False))
print("Test bin distribution:\n", pd.value_counts(y_binned[test_index], sort=False))

# Output the resulting dataframes
print("X_train:\n", X_train.head())
print("y_train:\n", y_train.head())
print("X_test:\n", X_test.head())
print("y_test:\n", y_test.head())

Train bin distribution:
 0.0    6
3.0    7
4.0    5
2.0    5
1.0    5
Name: count, dtype: int64
Test bin distribution:
 1.0    1
0.0    1
2.0    1
4.0    2
3.0    2
Name: count, dtype: int64
X_train:
                      gsv_wall_p100  gsv_building_p100  gsv_house_p100  \
site_name                                                               
Dhanmondi_lake_park         0.3000             1.3425          0.0025   
Chandrima_park              1.9925             0.2625          0.0000   
Aftabnagar                  3.6900            32.4275          0.2575   
Technical                   2.4050             8.3575          0.3650   
Tejgaon                     4.0325            16.0975          0.0000   

                     gsv_awning_p100  gsv_sky_p100  gsv_earth_p100  \
site_name                                                            
Dhanmondi_lake_park           0.0000       17.1325         20.7575   
Chandrima_park                0.0000       40.2650          1.3275   
Aftabna

<ipython-input-15-a1486702b7a8>:12: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  print("Train bin distribution:\n", pd.value_counts(y_binned[train_index], sort=False))
<ipython-input-15-a1486702b7a8>:13: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  print("Test bin distribution:\n", pd.value_counts(y_binned[test_index], sort=False))


In [ ]:
from scipy.stats import ks_2samp
ks_statistic, p_value = ks_2samp(y_train, y_test)

print(f"K-S statistic: {ks_statistic}")
print(f"P-value: {p_value}")

# Interpretation
alpha = 0.05
if p_value < alpha:
    print("The null hypothesis is rejected. The distributions of the train and test sets are different.")
else:
    print("The null hypothesis cannot be rejected. The distributions of the train and test sets are the same.")

K-S statistic: 0.25
P-value: 0.842454033893869
The null hypothesis cannot be rejected. The distributions of the train and test sets are the same.


In [ ]:
# Apply Kolmogorov-Smirnov test for each feature
ks_results = {}
failed_features = []
alpha = 0.05
for feature in X.columns:
    ks_statistic, p_value = ks_2samp(X_train[feature], X_test[feature])
    ks_results[feature] = {'ks_statistic': ks_statistic, 'p_value': p_value}
    if p_value < alpha:
        failed_features.append(feature)

# Print K-S test results for each feature
for feature, result in ks_results.items():
    print(f"Feature: {feature}")
    print(f"K-S statistic: {result['ks_statistic']}")
    print(f"P-value: {result['p_value']}")
    if result['p_value'] < alpha:
        print("The null hypothesis is rejected. The distributions of the train and test sets are different for this feature.\n")
    else:
        print("The null hypothesis cannot be rejected. The distributions of the train and test sets are the same for this feature.\n")

# Print features that failed the K-S test
if failed_features:
    print("Features that failed the K-S test (null hypothesis rejected):")
    for feature in failed_features:
        print(f"- {feature}")
else:
    print("All features passed the K-S test (null hypothesis not rejected).")

Feature: gsv_wall_p100
K-S statistic: 0.32142857142857145
P-value: 0.5556032252116138
The null hypothesis cannot be rejected. The distributions of the train and test sets are the same for this feature.

Feature: gsv_building_p100
K-S statistic: 0.4642857142857143
P-value: 0.14808640616728036
The null hypothesis cannot be rejected. The distributions of the train and test sets are the same for this feature.

Feature: gsv_house_p100
K-S statistic: 0.2857142857142857
P-value: 0.7050303962215889
The null hypothesis cannot be rejected. The distributions of the train and test sets are the same for this feature.

Feature: gsv_awning_p100
K-S statistic: 0.17857142857142858
P-value: 0.9883820703931286
The null hypothesis cannot be rejected. The distributions of the train and test sets are the same for this feature.

Feature: gsv_sky_p100
K-S statistic: 0.35714285714285715
P-value: 0.4252563751762208
The null hypothesis cannot be rejected. The distributions of the train and test sets are the same

In [ ]:
X_train_dropped = X_train.drop(columns=failed_features)
X_test_dropped = X_test.drop(columns=failed_features)

In [ ]:
trans = StandardScaler()
X_st = trans.fit_transform(X_train_dropped)
X_st = pd.DataFrame(X_st, columns=X_train_dropped.columns, index=X_train_dropped.index)

In [ ]:
X_test_st = trans.transform(X_test_dropped)
X_test_st = pd.DataFrame(X_test_st, columns=X_test_dropped.columns, index=X_test_dropped.index)

In [ ]:
X_Pred = pd.read_excel('/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/combined_total_v4.0.xlsx', index_col='OID_')

In [ ]:
col=X_st.columns
X_Pred_f1 = X_Pred[col]
X_Pred_f2= X_Pred_f1

In [ ]:
x_pred_st= trans.transform(X_Pred_f2)
x_pred_st = pd.DataFrame(x_pred_st, columns = X_Pred_f2.columns,index=X_Pred_f2.index)

In [ ]:
import itertools
import optuna
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor

# Feature list for PM2.5 dataset
feature_list = [
    ['gsv_tree_p750', 'RdclP_250', 'build_250', 'gsv_sidewalk_p250'],
    ['gsv_tree_p750', 'RdclP_250', 'gsv_plant_p500', 'build_250', 'gsv_sidewalk_p250'],
    ['gsv_tree_p750', 'RdclP_250', 'gsv_path_p500', 'build_250', 'gsv_sidewalk_p250']
]

# Function to run the optimization and evaluate the model
def evaluate_model(features, model_type):
    X_Train = X_st[features]
    X_test = X_test_st[features]

    def objective(trial):
        if model_type == 'XGBRegressor':
            learning_rate = trial.suggest_float('learning_rate', 0.001, 1.5)
            n_estimators = trial.suggest_int('n_estimators', 50, 2000)
            reg_alpha = trial.suggest_float('reg_alpha', 0.01, 100)
            reg_lambda = trial.suggest_float('reg_lambda', 0.01, 10)
            model = XGBRegressor(booster='gblinear',
                                 n_estimators=n_estimators,
                                 learning_rate=learning_rate,
                                 reg_alpha=reg_alpha,
                                 reg_lambda=reg_lambda,
                                 objective="reg:squarederror",
                                 seed=42)
        elif model_type == 'AdaBoostRegressor':
            n_estimators = trial.suggest_int("n_estimators", 50, 1000)
            learning_rate = trial.suggest_float("learning_rate", 0.001, 2.0)
            max_depth = trial.suggest_int("max_depth", 1, 50)
            estimator = DecisionTreeRegressor(max_depth=max_depth)
            model = AdaBoostRegressor(estimator=estimator,
                                      n_estimators=n_estimators,
                                      learning_rate=learning_rate,
                                      random_state=42)
        elif model_type == 'RandomForestRegressor':
            n_estimators = trial.suggest_int("n_estimators", 50, 2000)
            max_depth = trial.suggest_int("max_depth", 3, 50)
            min_samples_split = trial.suggest_int("min_samples_split", 2, 40)
            min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 40)
            model = RandomForestRegressor(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_split=min_samples_split,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=42)

        predicted = cross_val_predict(model, X_Train, y_train, cv=10)
        return mean_squared_error(y_train, predicted, squared=False)

    study = optuna.create_study(direction='minimize', study_name=f"{model_type}_RMSE")
    study.optimize(objective, n_trials=100)  # Adjust n_trials as necessary

    best_params = study.best_params
    if model_type == 'XGBRegressor':
        model = XGBRegressor(booster='gblinear',
                             learning_rate=best_params['learning_rate'],
                             n_estimators=best_params['n_estimators'],
                             reg_alpha=best_params['reg_alpha'],
                             reg_lambda=best_params['reg_lambda'],
                             objective="reg:squarederror",
                             seed=42)
    elif model_type == 'AdaBoostRegressor':
        estimator = DecisionTreeRegressor(max_depth=best_params['max_depth'])
        model = AdaBoostRegressor(estimator=estimator,
                                  n_estimators=best_params['n_estimators'],
                                  learning_rate=best_params['learning_rate'],
                                  random_state=42)
    elif model_type == 'RandomForestRegressor':
        model = RandomForestRegressor(n_estimators=best_params['n_estimators'],
                                      max_depth=best_params['max_depth'],
                                      min_samples_split=best_params['min_samples_split'],
                                      min_samples_leaf=best_params['min_samples_leaf'],
                                      random_state=42)

    model.fit(X_Train, y_train)

    pred_train = model.predict(X_Train)
    rmse_train = mean_squared_error(y_train, pred_train, squared=False)
    r2_train = r2_score(y_train, pred_train)

    predicted = cross_val_predict(model, X_Train, y_train, cv=10)
    rmse_cv = mean_squared_error(y_train, predicted, squared=False)
    r2_cv = r2_score(y_train, predicted)

    predicted_test = model.predict(X_test)
    rmse_test = mean_squared_error(y_test, predicted_test, squared=False)
    r2_test = r2_score(y_test, predicted_test)

    return {
        "features": features,
        "model": model_type,
        "rmse_train": rmse_train,
        "r2_train": r2_train,
        "rmse_cv": rmse_cv,
        "r2_cv": r2_cv,
        "rmse_test": rmse_test,
        "r2_test": r2_test
    }

# Evaluate the models using each set of features in the feature list and different model types
results = []
model_types = ['XGBRegressor', 'AdaBoostRegressor', 'RandomForestRegressor']

for feature_set in feature_list:
    for model_type in model_types:
        print(f"Evaluating combination: {feature_set} with model: {model_type}")
        try:
            result = evaluate_model(feature_set, model_type)
            results.append(result)
        except ValueError as e:
            if "Input contains NaN" in str(e):
                print(f"Skipping combination due to NaN values: {feature_set}")
            else:
                raise e  # Raise the error if it's not due to NaN values

# Create a DataFrame to store the results
results_df = pd.DataFrame(results)

# Display the results
print(results_df.head())

# Save the results to a CSV file
results_df.to_csv("/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/feature_selection_results_PM2_5.csv", index=False)


[I 2024-08-12 06:09:34,754] A new study created in memory with name: XGBRegressor_RMSE


Evaluating combination: ['gsv_tree_p750', 'RdclP_250', 'build_250', 'gsv_sidewalk_p250'] with model: XGBRegressor


[I 2024-08-12 06:09:35,780] Trial 0 finished with value: 14.238458410879076 and parameters: {'learning_rate': 0.261204184144057, 'n_estimators': 1287, 'reg_alpha': 54.38341466109939, 'reg_lambda': 0.7388854124203563}. Best is trial 0 with value: 14.238458410879076.
[I 2024-08-12 06:09:36,859] Trial 1 finished with value: 14.238458303713232 and parameters: {'learning_rate': 0.8258913126671202, 'n_estimators': 1440, 'reg_alpha': 50.52624409734642, 'reg_lambda': 6.920138382803655}. Best is trial 1 with value: 14.238458303713232.
[I 2024-08-12 06:09:37,926] Trial 2 finished with value: 14.238459633808136 and parameters: {'learning_rate': 0.5013610998995204, 'n_estimators': 1496, 'reg_alpha': 61.05583250651422, 'reg_lambda': 4.921884445660468}. Best is trial 1 with value: 14.238458303713232.
[I 2024-08-12 06:09:39,211] Trial 3 finished with value: 14.238458513923279 and parameters: {'learning_rate': 0.2377734545074513, 'n_estimators': 1889, 'reg_alpha': 78.46701598854484, 'reg_lambda': 5.25

Evaluating combination: ['gsv_tree_p750', 'RdclP_250', 'build_250', 'gsv_sidewalk_p250'] with model: AdaBoostRegressor


[I 2024-08-12 06:11:27,536] Trial 0 finished with value: 11.748860127081022 and parameters: {'n_estimators': 864, 'learning_rate': 0.9486823961183674, 'max_depth': 11}. Best is trial 0 with value: 11.748860127081022.
[I 2024-08-12 06:11:34,114] Trial 1 finished with value: 11.66802713156158 and parameters: {'n_estimators': 430, 'learning_rate': 1.072397908437408, 'max_depth': 44}. Best is trial 1 with value: 11.66802713156158.
[I 2024-08-12 06:11:43,294] Trial 2 finished with value: 11.815546659259692 and parameters: {'n_estimators': 482, 'learning_rate': 0.4978197156360334, 'max_depth': 21}. Best is trial 1 with value: 11.66802713156158.
[I 2024-08-12 06:11:44,415] Trial 3 finished with value: 11.821590417536889 and parameters: {'n_estimators': 69, 'learning_rate': 0.5566732344706931, 'max_depth': 36}. Best is trial 1 with value: 11.66802713156158.
[I 2024-08-12 06:11:45,912] Trial 4 finished with value: 11.386081729148845 and parameters: {'n_estimators': 91, 'learning_rate': 1.106115

...

Evaluating combination: ['gsv_tree_p750', 'RdclP_250', 'gsv_path_p500', 'build_250', 'gsv_sidewalk_p250'] with model: AdaBoostRegressor


[I 2024-08-12 07:42:30,736] Trial 0 finished with value: 11.722993279387795 and parameters: {'n_estimators': 319, 'learning_rate': 0.8973609631867233, 'max_depth': 41}. Best is trial 0 with value: 11.722993279387795.
[I 2024-08-12 07:42:34,673] Trial 1 finished with value: 12.324482023308612 and parameters: {'n_estimators': 190, 'learning_rate': 0.2225499991375207, 'max_depth': 20}. Best is trial 0 with value: 11.722993279387795.
[I 2024-08-12 07:42:41,979] Trial 2 finished with value: 12.24744871391589 and parameters: {'n_estimators': 471, 'learning_rate': 0.12433873690018535, 'max_depth': 31}. Best is trial 0 with value: 11.722993279387795.
[I 2024-08-12 07:42:44,484] Trial 3 finished with value: 12.01784387602749 and parameters: {'n_estimators': 133, 'learning_rate': 0.04506906909708131, 'max_depth': 33}. Best is trial 0 with value: 11.722993279387795.
[I 2024-08-12 07:42:48,593] Trial 4 finished with value: 11.992557215682925 and parameters: {'n_estimators': 179, 'learning_rate': 0

Evaluating combination: ['gsv_tree_p750', 'RdclP_250', 'gsv_path_p500', 'build_250', 'gsv_sidewalk_p250'] with model: RandomForestRegressor


[I 2024-08-12 07:55:33,749] Trial 0 finished with value: 14.233362369628923 and parameters: {'n_estimators': 1470, 'max_depth': 24, 'min_samples_split': 35, 'min_samples_leaf': 16}. Best is trial 0 with value: 14.233362369628923.
[I 2024-08-12 07:55:51,227] Trial 1 finished with value: 14.230122834785735 and parameters: {'n_estimators': 1299, 'max_depth': 14, 'min_samples_split': 12, 'min_samples_leaf': 12}. Best is trial 1 with value: 14.230122834785735.
[I 2024-08-12 07:56:04,350] Trial 2 finished with value: 11.88465967728535 and parameters: {'n_estimators': 976, 'max_depth': 19, 'min_samples_split': 5, 'min_samples_leaf': 5}. Best is trial 2 with value: 11.88465967728535.
[I 2024-08-12 07:56:16,161] Trial 3 finished with value: 14.233812258594233 and parameters: {'n_estimators': 833, 'max_depth': 5, 'min_samples_split': 38, 'min_samples_leaf': 21}. Best is trial 2 with value: 11.88465967728535.
[I 2024-08-12 07:56:39,954] Trial 4 finished with value: 14.234854506296546 and paramete

                                            features                  model  \
0  [gsv_tree_p750, RdclP_250, build_250, gsv_side...           XGBRegressor   
1  [gsv_tree_p750, RdclP_250, build_250, gsv_side...      AdaBoostRegressor   
2  [gsv_tree_p750, RdclP_250, build_250, gsv_side...  RandomForestRegressor   
3  [gsv_tree_p750, RdclP_250, gsv_plant_p500, bui...           XGBRegressor   
4  [gsv_tree_p750, RdclP_250, gsv_plant_p500, bui...      AdaBoostRegressor   

   rmse_train  r2_train    rmse_cv     r2_cv  rmse_test   r2_test  
0    8.619309  0.595864   9.764628  0.481326  14.006278  0.371067  
1    3.464102  0.934722   8.978148  0.561513  13.840933  0.385828  
2    8.855214  0.573439  11.466343  0.284792  16.520526  0.125003  
3    7.938387  0.657195   9.290777  0.530445  12.933447  0.463725  
4    3.256861  0.942299   9.890472  0.467871  13.763305  0.392698  
